# LIAR Fake News Detection

# Setup

## Install Libraries

In [ ]:
%pip install -r requirements.txt
!spacy download en_core_web_sm

## Import Libraries

In [ ]:
from nltk.corpus import stopwords
import nltk
import numpy as np
import spacy
import pandas as pd
import tensorflow as tf
import os.path
import pickle

nltk.download('stopwords')
nlp = spacy.load('en_core_web_sm')
# get the GPU device name
tf.config.list_physical_devices()

## Load Data

In [ ]:
train_data = pd.read_table('Data/train.tsv', names = ["id", "label", "statement", "subject", "speaker", "job", "state", "party", "barely-true", "false", "half-true", "mostly-true", "pants-fire", "venue"])
test_data = pd.read_table('Data/test.tsv', names = ["id", "label", "statement", "subject", "speaker", "job", "state", "party", "barely-true", "false", "half-true", "mostly-true", "pants-fire", "venue"])
valid_data = pd.read_table('Data/valid.tsv', names = ["id", "label", "statement", "subject", "speaker", "job", "state", "party", "barely-true", "false", "half-true", "mostly-true", "pants-fire", "venue"])

### Data Info

In [ ]:
print("Training Data Info:")
print(train_data.info())
print("Testing Data Info:")
print(test_data.info())
print("Validation Data Info:")
print(valid_data.info())
print(train_data.label.unique())
print(train_data.head())

# Preprocessing

## Convert Categorical Data to Numerical

### Output Labels

In [ ]:
# based on degree of truthfullness
y_label_dict = {"pants-fire" : 0, "false" : 1, "barely-true" : 2, "half-true" : 3, "mostly-true" : 4, "true" : 5}

train_data['output'] = train_data['label'].apply(lambda i: y_label_dict[i])
valid_data['output'] = valid_data['label'].apply(lambda i: y_label_dict[i])
test_data['output'] = test_data['label'].apply(lambda i: y_label_dict[i])


### Speakers

In [ ]:
# take number of top speakers to consider a parameter
no_speaker = 25
# based on the frequency of the label (only consider the top no_speaker speakers as relevent, after 20, rest have less than 50 data points, not relevent)
frequent_speakers = train_data['speaker'].value_counts().reset_index()[:no_speaker].to_dict()['speaker']
frequent_speakers = dict((v, k) for k, v in frequent_speakers.items())
print(frequent_speakers)

def convert_speaker_to_num(speaker):
  # speaker not in the top 20, assign it to the 21st category
  other = no_speaker
  if isinstance(speaker, str):
    if speaker in frequent_speakers:
      return frequent_speakers[speaker]
    else:
      return other
  else:
    return other

train_data['speaker_id'] = train_data['speaker'].apply(convert_speaker_to_num)
valid_data['speaker_id'] = valid_data['speaker'].apply(convert_speaker_to_num)
test_data['speaker_id'] = test_data['speaker'].apply(convert_speaker_to_num)
train_data['speaker_id'].value_counts()

### Job Titles

In [ ]:
# take number of top jobs to consider a parameter
no_jobs = 25
# based on the frequency of the label (only consider the top no_jobs speakers as relevent, after 20, rest have less than 50 data points, not relevent)
frequent_jobs = train_data['job'].value_counts().reset_index()[:no_jobs].to_dict()['job']
frequent_jobs = dict((v, k) for k, v in frequent_jobs.items())
print(frequent_jobs)

def convert_job_to_num(job):
  # job not in the top jobs, assign it to the last category
  other = no_jobs
  if isinstance(job, str):
    if job in frequent_jobs:
      return frequent_jobs[job]
    else:
      return other
  else:
    return other

train_data['job_id'] = train_data['job'].apply(convert_job_to_num)
valid_data['job_id'] = valid_data['job'].apply(convert_job_to_num)
test_data['job_id'] = test_data['job'].apply(convert_job_to_num)
train_data['job_id'].value_counts()

### Parties

In [ ]:
# take number of top parties to consider a parameter
no_party = 9
# based on the frequency of the label (only consider the top no_party speakers as relevent, after 20, rest have less than 50 data points, not relevent)
frequent_party = train_data['party'].value_counts().reset_index()[:no_party].to_dict()['party']
frequent_party = dict((v, k) for k, v in frequent_party.items())
print(frequent_party)

def convert_party_to_num(party):
  # party not in the top parties, assign it to the last category
  other = no_party
  if isinstance(party, str):
    if party in frequent_party:
      return frequent_party[party]
    else:
      return other
  else:
    return other

train_data['party_id'] = train_data['party'].apply(convert_party_to_num)
valid_data['party_id'] = valid_data['party'].apply(convert_party_to_num)
test_data['party_id'] = test_data['party'].apply(convert_party_to_num)
train_data['party_id'].value_counts()

### States

In [ ]:
# take number of top states to consider a parameter
no_state = 30
# based on the frequency of the label (only consider the top no_state speakers as relevent, after 20, rest have less than 50 data points, not relevent)
frequent_state = train_data['state'].value_counts().reset_index()[:no_state].to_dict()['state']
frequent_state = dict((v, k) for k, v in frequent_state.items())
print(frequent_state)

def convert_state_to_num(state):
  # state not in the top states, assign it to the last category
  other = no_state
  if isinstance(state, str):
    if state in frequent_state:
      return frequent_state[state]
    else:
      return other
  else:
    return other

train_data['state_id'] = train_data['state'].apply(convert_state_to_num)
valid_data['state_id'] = valid_data['state'].apply(convert_state_to_num)
test_data['state_id'] = test_data['state'].apply(convert_state_to_num)
train_data['state_id'].value_counts()

### Subjects

In [ ]:
# take number of top subjects to consider a parameter
no_subject = 30
# based on the frequency of the label (only consider the top no_subject speakers as relevent, after 20, rest have less than 50 data points, not relevent)
frequent_subject = train_data['subject'].value_counts().reset_index()[:no_subject].to_dict()['subject']
frequent_subject = dict((v, k) for k, v in frequent_subject.items())
print(frequent_subject)

def convert_subject_to_num(subject):
  # subject not in the top subjects, assign it to the last category
  other = no_subject
  if isinstance(subject, str):
    if subject in frequent_subject:
      return frequent_subject[subject]
    else:
      return other
  else:
    return other

train_data['subject_id'] = train_data['subject'].apply(convert_subject_to_num)
valid_data['subject_id'] = valid_data['subject'].apply(convert_subject_to_num)
test_data['subject_id'] = test_data['subject'].apply(convert_subject_to_num)
train_data['subject_id'].value_counts()

### Venues

In [ ]:
# take number of top venues to consider a parameter
no_venue = 30
# based on the frequency of the label (only consider the top no_venue speakers as relevent, after 20, rest have less than 50 data points, not relevent)
frequent_venue = train_data['venue'].value_counts().reset_index()[:no_venue].to_dict()['venue']
frequent_venue = dict((v, k) for k, v in frequent_venue.items())
print(frequent_venue)

def convert_venue_to_num(venue):
  # venue not in the top venues, assign it to the last category
  other = no_venue
  if isinstance(venue, str):
    if venue in frequent_venue:
      return frequent_venue[venue]
    else:
      return other
  else:
    return other

train_data['venue_id'] = train_data['venue'].apply(convert_venue_to_num)
valid_data['venue_id'] = valid_data['venue'].apply(convert_venue_to_num)
test_data['venue_id'] = test_data['venue'].apply(convert_venue_to_num)
train_data['venue_id'].value_counts()

## Tokenizing Content

### Word Frequency Tokenization

In [ ]:
vocab_dict = {}
if not os.path.exists('vocab_dict.pkl'):
  tokenizer = tf.keras.preprocessing.text.Tokenizer()
  tokenizer.fit_on_texts(train_data['statement'])
  vocab_dict = tokenizer.word_index
  pickle.dump(vocab_dict, open('vocab_dict.pkl', 'wb'))
else:
  vocab_dict = pickle.load(open('vocab_dict.pkl', 'rb'))

def convert_statement_to_vec(statement):
  stmnt = ''.join(word for word in statement.split() if word not in stopwords.words('english'))
  text = tf.keras.preprocessing.text.text_to_word_sequence(stmnt)
  return [vocab_dict[word] for word in text if word in vocab_dict]

train_data['statement_freq'] = train_data['statement'].apply(convert_statement_to_vec)
valid_data['statement_freq'] = valid_data['statement'].apply(convert_statement_to_vec)
test_data['statement_freq'] = test_data['statement'].apply(convert_statement_to_vec)

### Part of Speech Tagging

In [ ]:
"""
pos_tags = {'ADJ': 'adjective', 'ADP': 'adposition', 'ADV': 'adverb',
            'AUX': 'auxiliary verb', 'CONJ': 'coordinating conjunction',
            'DET': 'determiner', 'INTJ': 'interjection', 'NOUN': 'noun',
            'NUM': 'numeral', 'PART': 'particle', 'PRON': 'pronoun',
            'PROPN': 'proper noun', 'PUNCT': 'punctuation', 'X': 'other',
            'SCONJ': 'subord conjunction', 'SYM': 'symbol', 'VERB': 'verb'}
"""
# create a dictionary to convert the pos tags to numbers, arbitrary
pos_dict = {'NOUN' : 0, 'VERB' : 1, 'ADP' : 2, 'PROPN' : 3, 'PUNCT' : 4,
            'DET' : 5, 'ADJ' : 6, 'NUM' : 7, 'ADV' : 8, 'PRON' : 9}
other = len(pos_dict.values())  # fpr all other pos tags

def convert_sentence_to_pos(sentence: str):
  doc = nlp(sentence)
  return [pos_dict.get(token.pos_, other) for token in doc]

train_data['statement_pos'] = train_data['statement'].apply(convert_sentence_to_pos)
valid_data['statement_pos'] = valid_data['statement'].apply(convert_sentence_to_pos)
test_data['statement_pos'] = test_data['statement'].apply(convert_sentence_to_pos)

### Dependency Parsing

In [ ]:
"""
all dependencies:
dep_dict = {'ACL' : 0, 'ACOMP' : 1, 'ADVCL' : 2, 'ADVMOD' : 3, 'AGENT' : 4,
            'AMOD' : 5, 'APPOS' : 6, 'ATTR' : 7, 'AUX' : 8, 'AUXPASS' : 9,
            'CASE' : 10, 'CC' : 11, 'CCOMP' : 12, 'COMPOUND' : 13, 'CONJ' : 14,
            'CSUBJ' : 15, 'CSUBJPASS' : 16, 'DATIVE' : 17, 'DEP' : 18,
            'DET' : 19, 'DOBJ' : 20, 'EXPL' : 21, 'INTJ' : 22, 'MARK' : 23,
            'META' : 24, 'NEG' : 25, 'NOUNMOD' : 26, 'NPMOD' : 27, 'NSUBJ' : 28,
            'NSUBJPASS' : 29, 'NUMMOD' : 30, 'OPRD' : 31, 'PARATAXIS' : 32,
            'PCOMP' : 33, 'POBJ' : 34, 'POSS' : 35, 'PRECONJ' : 36, 'PREDET' : 37,
            'PREP' : 38, 'PRT' : 39, 'PUNCT' : 40, 'QUANTMOD' : 41,
            'RELCL' : 42, 'ROOT' : 43, 'XCOMP' : 44}
"""
# create a dictionary to convert the dep tags to numbers, arbitrary
dep_dict = {'punct' : 0, 'prep' : 1, 'pobj' : 2, 'compound' : 3, 'det' : 4,
            'nsubj' : 5, 'ROOT' : 6, 'amod' : 7, 'dobj' : 8, 'aux' : 9}
other = len(dep_dict.values())  # for all other dep tags

def convert_sentence_to_dep(sentence):
  doc = nlp(sentence)
  return [dep_dict.get(token.dep_, other) for token in doc]

train_data['statement_dep'] = train_data['statement'].apply(convert_sentence_to_dep)
valid_data['statement_dep'] = valid_data['statement'].apply(convert_sentence_to_dep)
test_data['statement_dep'] = test_data['statement'].apply(convert_sentence_to_dep)

In [ ]:
train_data.head()

## Embeddings
We use the pretrained GloVe embeddings to convertwords into embeddings

In [ ]:
embed_dim = 100
embeddings = {}
word = ''
try:
  with open('glove.6B.100d.txt', 'r') as f:
    for line in f:
      values = line.split()
      word = values[0].lower()
      embeddings[word] = np.asarray(values[1:], dtype='float32')
except FileNotFoundError:
  print('File glove.6B.100d.txt was not found in this directory')
  print('Get the file from the references provided in README.md')
  raise FileNotFoundError
print(len(embeddings), ": Embeddings loaded")
print(embed_dim, ": Embedding dimension")

num_words = len(vocab_dict) + 1
embed_matrix = np.zeros((num_words, embed_dim))
for word, i in vocab_dict.items():
  embed_vector = embeddings.get(word)
  if embed_vector is not None:
    embed_matrix[i] = embed_vector

pos_embeddings = np.identity(max(pos_dict.values())+1, dtype=int)
dep_embeddings = np.identity(max(dep_dict.values())+1, dtype=int)

## Global Hyperparameters

In [ ]:
vocab_length = len(vocab_dict.keys())
lstm_size = 100
num_steps = 15
num_epochs = 30
batch_size = 40

#Hyperparams for CNN
kernel_sizes = [3,3,3]
filter_size = 128

#Meta data related hyper params
num_party = len(train_data.party_id.unique())
num_state = len(train_data.state_id.unique())
num_venue = len(train_data.venue_id.unique())
num_job = len(train_data.job_id.unique())
num_sub = len(train_data.subject_id.unique())
num_speaker = len(train_data.speaker_id.unique())

## Prepare Sentence Info (Padding)

In [ ]:
X_train = train_data['statement_freq']
X_val = valid_data['statement_freq']
X_test = test_data['statement_freq']

Y_train = tf.keras.utils.to_categorical(train_data['output'], num_classes=6)
Y_val = tf.keras.utils.to_categorical(valid_data['output'], num_classes=6)
Y_test = list(test_data['output'])

X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=num_steps, padding='post', truncating='post')
X_val = tf.keras.preprocessing.sequence.pad_sequences(X_val, maxlen=num_steps, padding='post', truncating='post')
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=num_steps, padding='post', truncating='post')

X_train_pos = train_data['statement_pos']
X_val_pos = valid_data['statement_pos']
X_test_pos = test_data['statement_pos']

X_train_pos = tf.keras.preprocessing.sequence.pad_sequences(X_train_pos, maxlen=num_steps, padding='post', truncating='post')
X_val_pos = tf.keras.preprocessing.sequence.pad_sequences(X_val_pos, maxlen=num_steps, padding='post', truncating='post')
X_test_pos = tf.keras.preprocessing.sequence.pad_sequences(X_test_pos, maxlen=num_steps, padding='post', truncating='post')

X_train_dep = train_data['statement_dep']
X_val_dep = valid_data['statement_dep']
X_test_dep = test_data['statement_dep']

X_train_dep = tf.keras.preprocessing.sequence.pad_sequences(X_train_dep, maxlen=num_steps, padding='post', truncating='post')
X_val_dep = tf.keras.preprocessing.sequence.pad_sequences(X_val_dep, maxlen=num_steps, padding='post', truncating='post')
X_test_dep = tf.keras.preprocessing.sequence.pad_sequences(X_test_dep, maxlen=num_steps, padding='post', truncating='post')

## Meta Data Preparation

In [ ]:
party_train = tf.keras.utils.to_categorical(train_data['party_id'], num_classes=num_party)
party_val = tf.keras.utils.to_categorical(valid_data['party_id'], num_classes=num_party)
party_test = tf.keras.utils.to_categorical(test_data['party_id'], num_classes=num_party)

state_train = tf.keras.utils.to_categorical(train_data['state_id'], num_classes=num_state)
state_val = tf.keras.utils.to_categorical(valid_data['state_id'], num_classes=num_state)
state_test = tf.keras.utils.to_categorical(test_data['state_id'], num_classes=num_state)

venue_train = tf.keras.utils.to_categorical(train_data['venue_id'], num_classes=num_venue)
venue_val = tf.keras.utils.to_categorical(valid_data['venue_id'], num_classes=num_venue)
venue_test = tf.keras.utils.to_categorical(test_data['venue_id'], num_classes=num_venue)

job_train = tf.keras.utils.to_categorical(train_data['job_id'], num_classes=num_job)
job_val = tf.keras.utils.to_categorical(valid_data['job_id'], num_classes=num_job)
job_test = tf.keras.utils.to_categorical(test_data['job_id'], num_classes=num_job)

subject_train = tf.keras.utils.to_categorical(train_data['subject_id'], num_classes=num_sub)
subject_val = tf.keras.utils.to_categorical(valid_data['subject_id'], num_classes=num_sub)
subject_test = tf.keras.utils.to_categorical(test_data['subject_id'], num_classes=num_sub)

speaker_train = tf.keras.utils.to_categorical(train_data['speaker_id'], num_classes=num_speaker)
speaker_val = tf.keras.utils.to_categorical(valid_data['speaker_id'], num_classes=num_speaker)
speaker_test = tf.keras.utils.to_categorical(test_data['speaker_id'], num_classes=num_speaker)

X_train_meta = np.hstack((party_train, state_train, venue_train, job_train, subject_train, speaker_train))
X_val_meta = np.hstack((party_val, state_val, venue_val, job_val, subject_val, speaker_val))
X_test_meta = np.hstack((party_test, state_test, venue_test, job_test, subject_test, speaker_test))

## Check Matrix Shapes

In [ ]:
print(X_train_meta.shape, X_val_meta.shape, X_test_meta.shape)
print(X_train.shape, X_val.shape, X_test.shape)
print(Y_train.shape, Y_val.shape)
print(X_train_pos.shape, X_val_pos.shape, X_test_pos.shape)
print(X_train_dep.shape, X_val_dep.shape, X_test_dep.shape)

## Check Processed Data

In [ ]:
train_data.head()

# Defining Functions

## Train Function

In [ ]:
def train(model: tf.keras.models.Model, model_file_name: str, use_pos = False, use_meta = False, use_dep = False):
  sgd = tf.keras.optimizers.SGD(learning_rate=0.025, clipvalue=0.3, nesterov=True)
  # adam = tf.keras.optimizers.Adam(lr=0.000075, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
  model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
  tb = tf.keras.callbacks.TensorBoard()
  csv_logger = tf.keras.callbacks.CSVLogger('train.log')
  filepath = model_file_name + '_weights.keras'
  checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_categorical_accuracy', verbose=1, save_best_only=True, mode='max')

  train_input = [X_train]
  valid_input = [X_val]
  if use_pos:
    train_input.append(X_train_pos)
    valid_input.append(X_val_pos)
  if use_dep:
    train_input.append(X_train_dep)
    valid_input.append(X_val_dep)
  if use_meta:
    train_input.append(X_train_meta)
    valid_input.append(X_val_meta)
  model.fit(train_input, [Y_train], epochs=num_epochs, batch_size=batch_size, validation_data=(valid_input, [Y_val]), callbacks=[tb, csv_logger, checkpoint])

## Test Function

In [ ]:
def test(model_file_name: str, use_pos = False, use_meta = False, use_dep = False):
  model: tf.keras.models.Model = tf.keras.models.load_model(model_file_name + '_weights.keras')
  input = [X_test]
  if use_pos:
    input.append(X_test_pos)
  if use_dep:
    input.append(X_test_dep)
  if use_meta:
    input.append(X_test_meta)
  predictions = model.predict(input, batch_size=batch_size, verbose=1)
  n = len(predictions)

  tp = tn = fp = fn = 0
  for p in range(n):
    if np.argmax(predictions[p]) == Y_test[p]:
      if Y_test[p]  >= 3:
        tp += 1
      else:
        tn += 1
    else:
      if Y_test[p] >= 3:
        fn += 1
      else:
        fp += 1
  print(n == len(Y_test))
  correct = np.sum(np.argmax(predictions, axis=1) == Y_test)
  print("Correctly predicted: ", correct, "out of", n)
  print("Accuracy: ", correct*100/n)
  pickle.dump(predictions, open(model_file_name + '_predictions.pkl', 'wb'))

  print("True Positive: ", tp)
  print("True Negative: ", tn)
  print("False Positive: ", fp)
  print("False Negative: ", fn)

# Building Models

## Shared Hyperparameters

In [ ]:
use_pos = False
use_meta = True
use_dep = True

## CNN

In [ ]:
statement_input = tf.keras.layers.Input(shape=(num_steps,), dtype='int32', name='main_input')
x_stmt = tf.keras.layers.Embedding(vocab_length+1, embed_dim, weights=[embed_matrix], trainable=False)(statement_input)

pos_input = tf.keras.layers.Input(shape=(num_steps,), dtype='int32', name='pos_input')
x_pos = tf.keras.layers.Embedding(max(pos_dict.values())+1, max(pos_dict.values())+1, weights=[pos_embeddings], trainable=False)(pos_input)

dep_input = tf.keras.layers.Input(shape=(num_steps,), dtype='int32', name='dep_input')
x_dep = tf.keras.layers.Embedding(max(dep_dict.values())+1, max(dep_dict.values())+1, weights=[dep_embeddings], trainable=False)(dep_input)

meta_input = tf.keras.layers.Input(shape=(X_train_meta.shape[1],), name='aux_input')
x_meta = tf.keras.layers.Dense(64, activation='relu')(meta_input)

kernel_stmt = []
kernel_pos = []
kernel_dep = []
for kernel in kernel_sizes:
  x_1 = tf.keras.layers.Conv1D(filter_size, kernel)(x_stmt)
  x_1 = tf.keras.layers.GlobalMaxPooling1D()(x_1)
  kernel_stmt.append(x_1)

  x_2 = tf.keras.layers.Conv1D(filter_size, kernel)(x_pos)
  x_2 = tf.keras.layers.GlobalMaxPooling1D()(x_2)
  kernel_pos.append(x_2)

  x_3 = tf.keras.layers.Conv1D(filter_size, kernel)(x_dep)
  x_3 = tf.keras.layers.GlobalMaxPooling1D()(x_3)
  kernel_dep.append(x_3)

conv_in1 = tf.keras.layers.concatenate(kernel_stmt)
conv_in1 = tf.keras.layers.Dropout(0.6)(conv_in1)
conv_in1 = tf.keras.layers.Dense(128, activation='relu')(conv_in1)

conv_in2 = tf.keras.layers.concatenate(kernel_pos)
conv_in2 = tf.keras.layers.Dropout(0.6)(conv_in2)
conv_in2 = tf.keras.layers.Dense(128, activation='relu')(conv_in2)

conv_in3 = tf.keras.layers.concatenate(kernel_dep)
conv_in3 = tf.keras.layers.Dropout(0.6)(conv_in3)
conv_in3 = tf.keras.layers.Dense(128, activation='relu')(conv_in3)

lays = [conv_in1]
if use_pos:
  lays.append(conv_in2)
if use_dep:
  lays.append(conv_in3)
if use_meta:
  lays.append(x_meta)
x = tf.keras.layers.concatenate(lays)

main_output = tf.keras.layers.Dense(6, activation='softmax', name='main_output')(x)
inputs = [statement_input]
if use_pos:
  inputs.append(pos_input)
if use_dep:
  inputs.append(dep_input)
if use_meta:
  inputs.append(meta_input)
model_cnn = tf.keras.models.Model(inputs=inputs, outputs=[main_output])
print(model_cnn.summary())

## LSTM

In [ ]:
model_lstm = tf.keras.models.Sequential()
hidden_size = embed_dim
model_lstm.add(tf.keras.layers.Embedding(vocab_length+1, hidden_size))
model_lstm.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(hidden_size)))
model_lstm.add(tf.keras.layers.Dense(6, activation='softmax'))

statement_input = tf.keras.layers.Input(shape=(num_steps,), dtype='int32', name='main_input')
x_stmt = tf.keras.layers.Embedding(vocab_length+1, embed_dim, weights=[embed_matrix], trainable=False)(statement_input)
lstm_in = tf.keras.layers.LSTM(lstm_size, dropout=0.2)(x_stmt)

pos_input = tf.keras.layers.Input(shape=(num_steps,), dtype='int32', name='pos_input')
x_pos = tf.keras.layers.Embedding(max(pos_dict.values())+1, max(pos_dict.values())+1, weights=[pos_embeddings], trainable=False)(pos_input)
lstm_in2 = tf.keras.layers.LSTM(lstm_size, dropout=0.2)(x_pos)

dep_input = tf.keras.layers.Input(shape=(num_steps,), dtype='int32', name='dep_input')
x_dep = tf.keras.layers.Embedding(max(dep_dict.values())+1, max(dep_dict.values())+1, weights=[dep_embeddings], trainable=False)(dep_input)
lstm_in3 = tf.keras.layers.LSTM(lstm_size, dropout=0.2)(x_dep)

meta_input = tf.keras.layers.Input(shape=(X_train_meta.shape[1],), name='aux_input')
x_meta = tf.keras.layers.Dense(64, activation='relu')(meta_input)

lays = [lstm_in]
if use_pos:
  lays.append(lstm_in2)
if use_dep:
  lays.append(lstm_in3)
if use_meta:
  lays.append(x_meta)
x = tf.keras.layers.concatenate(lays)

main_output = tf.keras.layers.Dense(6, activation='softmax', name='main_output')(x)
inputs = [statement_input]
if use_pos:
  inputs.append(pos_input)
if use_dep:
  inputs.append(dep_input)
if use_meta:
  inputs.append(meta_input)
model_lstm = tf.keras.models.Model(inputs=inputs, outputs=[main_output])
print(model_lstm.summary())

# Training Models

## CNN

In [ ]:
train(model_cnn, 'cnn', use_pos, use_meta, use_dep)

## LSTM

In [ ]:
train(model_lstm,'lstm', use_pos, use_meta, use_dep)

# Testing Models

## CNN

In [ ]:
test('cnn', use_pos, use_meta, use_dep)

## LSTM

In [ ]:
test('lstm', use_pos, use_meta, use_dep)